Germ Game
---
Two players against each other

In [1]:
import numpy as np
import pickle
import sys
from copy import deepcopy

In [2]:
BOARD_ROWS = 7
BOARD_COLS = 7

### Board State
---
Reflect & Judge the state

2 players p1 and p2; p1 uses symbol 1 and p2 uses symbol 2, vacancy as 0

In [3]:
class State:
    def __init__(self, p1, p2):
        self.board = np.zeros((BOARD_ROWS, BOARD_COLS))
        self.board[0, 0] = self.board[BOARD_ROWS-1, BOARD_COLS-1] = 1
        self.board[BOARD_ROWS-1, 0] = self.board[0, BOARD_COLS-1] = 2
        self.p1 = p1
        self.p2 = p2
        self.isEnd = False
        self.boardHash = None
        # init p1 plays first
        self.playerSymbol = 1
    
    # get unique hash of current board state
    def getHash(self, position):
        next_board = deepcopy(self.board)
        p = position.copy()
        
        if p[2]-p[0]<0:
            for i in range(3):
                for j in range(3):
                    next_board[i][j],next_board[6-i][j] = next_board[6-i][j],next_board[i][j]
            p[0]=6-p[0]
            p[2]=6-p[2]

        if p[3]-p[1]<0:
            for i in range(7):
                for j in range(7):
                    next_board[i][j],next_board[i][6-j] = next_board[i][6-j],next_board[i][j]
            p[1]=6-p[1]
            p[3]=6-p[3]

        if p[2]-p[0]<p[3]-p[1]:
            for i in range(7):
                for j in range(i+1,7):
                    next_board[i][j],next_board[j][i] = next_board[j][i],next_board[i][j]
            p[0],p[1] = p[1],p[0]
            p[2],p[3] = p[3],p[2]
            
        s1 = s2 = s3 = 0
        e2 = e3 = 0
        shape = 0

        for dx in range(-3,4):
            for dy in range(-3,4):
                if not dx and not dy:
                    continue
                if p[0]+dx<0 or p[0]+dx>=BOARD_ROWS or p[1]+dy<0 or p[1]+dy>=BOARD_COLS:
                    continue

                if next_board[p[0]+dx][p[1]+dy]!=3-self.playerSymbol:
                    continue
                if max(abs(dx),abs(dy))==1:
                    s1=1
                if max(abs(dx),abs(dy))==2:
                    s2=1
                if max(abs(dx),abs(dy))==3:
                    s3=1

        for dx in range(-3,4):
            for dy in range(-3,4):
                if abs(dx)<=1 and abs(dy)<=1:
                    continue
                if p[2]+dx<0 or p[2]+dx>=BOARD_ROWS or p[3]+dy<0 or p[3]+dy>=BOARD_COLS:
                    continue

                if next_board[p[2]+dx][p[3]+dy]!=3-self.playerSymbol:
                    continue
                if max(abs(dx),abs(dy))==2:
                    e2=1
                if max(abs(dx),abs(dy))==3:
                    e3=1

        for dx in range(-1,2):
            for dy in range(-1,2):
                if not dx and not dy:
                    continue

                shape = shape<<1
                if p[2]+dx<0 or p[2]+dx>=BOARD_ROWS or p[3]+dy<0 or p[3]+dy>=BOARD_COLS:
                    shape = shape|1
                elif next_board[p[2]+dx][p[3]+dy]:
                    shape = shape|1

        # print(typ,s1,s2,s3,e2,e3,shape)

        if(p[3]-p[1]<2):
            return ((p[3]-p[1])<<12)+((s1|s2)<<11)+(s3<<10)+(e2<<9)+(e3<<8)+shape+1
        return ((p[3]-p[1])<<12)+(s1<<11)+(s2<<10)+(e2<<9)+(e3<<8)+shape+1
    
    def cantmove(self):
        for i in range(BOARD_ROWS):
            for j in range(BOARD_COLS):
                if self.board[i, j] == 0:
                    self.board[i, j] = 3 - self.playerSymbol
        return None
    
    def winner(self):
        # end
        if sum(map(sum,list(map(lambda row: list(map(lambda x: abs(2*x-3) if x!=0 else 0, row)), self.board)))) == BOARD_ROWS*BOARD_COLS:
            self.isEnd = True
            if sum(map(sum,list(map(lambda row: list(map(lambda x: 2*x-3 if x!=0 else 0, row)), self.board)))) < 0:
                return 1
            else:
                return 2
        # not end
        self.isEnd = False
        return None
    
    def availablePositions(self):
        positions = []
        for i in range(BOARD_ROWS):
            for j in range(BOARD_COLS):
                if self.board[i, j] == self.playerSymbol:
                    for ii in range(-2,3):
                        for jj in range(-2,3):
                            if ii == 0 and jj == 0:
                                continue
                            if i + ii < 0 or i + ii >= BOARD_ROWS or j + jj < 0 or j + jj >= BOARD_COLS:
                                continue
                            if self.board[i + ii, j + jj] == 0:
                                positions.append([i, j, i + ii, j + jj])  # need to be tuple
        return positions
    
    def updateState(self, position):
        ii = position[2] - position[0]
        jj = position[3] - position[1]
        if max(abs(ii), abs(jj)) == 2:
            self.board[position[0], position[1]] = 0
        
        dx1 = [-1, -1, -1, 0, 0, 1, 1, 1]
        dy1 = [-1, 0, 1, -1, 1, -1, 0, 1]
        i, j = position[2:4]
        self.board[i, j] = self.playerSymbol
        for ii, jj in zip(dx1, dy1):
            if i + ii < 0 or i + ii >= BOARD_ROWS or j + jj < 0 or j + jj >= BOARD_COLS:
                continue
            if self.board[i + ii, j + jj] == 3 - self.playerSymbol:
                self.board[i + ii, j + jj] = self.playerSymbol
            
        # switch to another player
        self.playerSymbol = 3 - self.playerSymbol
    
    # only when game ends
    def giveReward(self):
        result = self.winner()
        # backpropagate reward
        """self.p1.feedReward(-result)
        self.p2.feedReward(result)"""
        if result == 1:
            self.p1.feedReward(1)
            self.p2.feedReward(0)
        else:
            self.p1.feedReward(0)
            self.p2.feedReward(1)
    
    # board reset
    def reset(self):
        self.board = np.zeros((BOARD_ROWS, BOARD_COLS))
        self.board[0, 0] = self.board[BOARD_ROWS-1, BOARD_COLS-1] = 1
        self.board[BOARD_ROWS-1, 0] = self.board[0, BOARD_COLS-1] = 2
        self.boardHash = None
        self.isEnd = False
        self.playerSymbol = 2
    
    def play(self, rounds=100):
        for i in range(rounds):
            if i%10 == 0:
                print("Rounds {}".format(i))
            while not self.isEnd:
                # Player 1
                positions = self.availablePositions()
                if not positions:
                    self.cantmove()
                else:
                    p1_action = self.p1.chooseAction(positions, self.board, self.playerSymbol)
                    # take action and upate board state
                    self.updateState(p1_action)
                    # print("yay")
                    # print(positions)
                    # print(self.playerSymbol)
                    # print(p1_action)
                # self.showBoard()
                board_hash = self.getHash(p1_action)
                self.p1.addState(board_hash)
                # check board status if it is end
                win = self.winner()
                if win is not None:
                    # self.showBoard()
                    # ended with p1 either win or draw
                    self.giveReward()
                    self.p1.reset()
                    self.p2.reset()
                    self.reset()
                    break

                else:
                    # Player 2
                    positions = self.availablePositions()
                    if not positions:
                        self.cantmove()
                    else:
                        p2_action = self.p2.chooseAction(positions, self.board, self.playerSymbol)
                        self.updateState(p2_action)
                        # print("aya")
                        # print(positions)
                        # print(self.playerSymbol)
                        # print(p2_action)
                    # self.showBoard()
                    board_hash = self.getHash(p2_action)
                    self.p2.addState(board_hash)
                    
                    win = self.winner()
                    if win is not None:
                        # self.showBoard()
                        # ended with p2 either win or draw
                        self.giveReward()
                        self.p1.reset()
                        self.p2.reset()
                        self.reset()
                        break
    
    # play with human
    def play2(self):
        while not self.isEnd:
            # Player 1
            positions = self.availablePositions()
            if not positions:
                self.cantmove()
            else:
                # human
                # p1_action = self.p1.chooseAction(positions)
                # computer
                p1_action = self.p1.chooseAction(positions, self.board, self.playerSymbol)
                # take action and upate board state
                self.updateState(p1_action)
            self.showBoard()
            # check board status if it is end
            win = self.winner()
            if win is not None:
                if win == 1:
                    print(self.p1.name, "wins!")
                else:
                    print(self.p2.name, "wins!")
                self.reset()
                break

            else:
                # Player 2
                positions = self.availablePositions()
                if not positions:
                    self.cantmove()
                else:
                    # human
                    # p2_action = self.p2.chooseAction(positions)
                    # computer
                    p2_action = self.p2.chooseAction(positions, self.board, self.playerSymbol)
                    # take action and upate board state
                    self.updateState(p2_action)
                self.showBoard()
                win = self.winner()
                if win is not None:
                    if win == 1:
                        print(self.p1.name, "wins!")
                    else:
                        print(self.p2.name, "wins!")
                    print()
                    self.reset()
                    break

    def showBoard(self):
        # p1: x  p2: o
        for i in range(0, BOARD_ROWS):
            print('------------------------------')
            out = '| '
            for j in range(0, BOARD_COLS):
                if self.board[i, j] == 1:
                    token = 'x'
                if self.board[i, j] == 2:
                    token = 'o'
                if self.board[i, j] == 0:
                    token = ' '
                out += token + ' | '
            print(out)
        print('------------------------------')    

In [4]:
class Player:
    def __init__(self, name, exp_rate=0.1):
        self.name = name
        self.states = []  # record all positions taken
        self.lr = 0.2
        self.exp_rate = exp_rate
        self.decay_gamma = 0.95
        self.states_value = {}  # state -> value
    
    def getHash(self, current_board, position, symbol):
        next_board = deepcopy(current_board)
        p = position.copy()
        
        if p[2]-p[0]<0:
            for i in range(3):
                for j in range(3):
                    next_board[i][j],next_board[6-i][j] = next_board[6-i][j],next_board[i][j]
            p[0]=6-p[0]
            p[2]=6-p[2]

        if p[3]-p[1]<0:
            for i in range(7):
                for j in range(7):
                    next_board[i][j],next_board[i][6-j] = next_board[i][6-j],next_board[i][j]
            p[1]=6-p[1]
            p[3]=6-p[3]

        if p[2]-p[0]<p[3]-p[1]:
            for i in range(7):
                for j in range(i+1,7):
                    next_board[i][j],next_board[j][i] = next_board[j][i],next_board[i][j]
            p[0],p[1] = p[1],p[0]
            p[2],p[3] = p[3],p[2]
            
        s1 = s2 = s3 = 0
        e2 = e3 = 0
        shape = 0

        for dx in range(-3,4):
            for dy in range(-3,4):
                if not dx and not dy:
                    continue
                if p[0]+dx<0 or p[0]+dx>=BOARD_ROWS or p[1]+dy<0 or p[1]+dy>=BOARD_COLS:
                    continue

                if next_board[p[0]+dx][p[1]+dy]!=3-symbol:
                    continue
                if max(abs(dx),abs(dy))==1:
                    s1=1
                if max(abs(dx),abs(dy))==2:
                    s2=1
                if max(abs(dx),abs(dy))==3:
                    s3=1

        for dx in range(-3,4):
            for dy in range(-3,4):
                if abs(dx)<=1 and abs(dy)<=1:
                    continue
                if p[2]+dx<0 or p[2]+dx>=BOARD_ROWS or p[3]+dy<0 or p[3]+dy>=BOARD_COLS:
                    continue

                if next_board[p[2]+dx][p[3]+dy]!=3-symbol:
                    continue
                if max(abs(dx),abs(dy))==2:
                    e2=1
                if max(abs(dx),abs(dy))==3:
                    e3=1

        for dx in range(-1,2):
            for dy in range(-1,2):
                if not dx and not dy:
                    continue

                shape = shape<<1
                if p[2]+dx<0 or p[2]+dx>=BOARD_ROWS or p[3]+dy<0 or p[3]+dy>=BOARD_COLS:
                    shape = shape|1
                elif next_board[p[2]+dx][p[3]+dy]:
                    shape = shape|1

        # print(typ,s1,s2,s3,e2,e3,shape)

        if(p[3]-p[1]<2):
            return ((p[3]-p[1])<<12)+((s1|s2)<<11)+(s3<<10)+(e2<<9)+(e3<<8)+shape+1
        return ((p[3]-p[1])<<12)+(s1<<11)+(s2<<10)+(e2<<9)+(e3<<8)+shape+1
    
    def chooseAction(self, positions, current_board, symbol):
        if np.random.uniform(0, 1) <= self.exp_rate:
            # take random action
            idx = np.random.choice(len(positions))
            action = positions[idx]
        else:
            value_max = -999
            for p in positions:
                # print("p", p)
                boardHash = self.getHash(current_board, p, symbol)
                # print("boardHash", boardHash, self.states_value.get(boardHash))
                value = 0 if self.states_value.get(boardHash) is None else self.states_value.get(boardHash)
                # print("value", value, value_max)
                if value >= value_max:
                    value_max = value
                    action = p
        # print("{} takes action {}".format(self.name, action))
                    # print("action", action)
        return action
    
    # append a hash state
    def addState(self, state):
        self.states.append(state)
    
    # at the end of game, backpropagate and update states value
    def feedReward(self, reward):
        for st in reversed(self.states):
            if self.states_value.get(st) is None:
                self.states_value[st] = 0
            self.states_value[st] += self.lr*(self.decay_gamma*reward - self.states_value[st])
            reward = self.states_value[st]
            
    def reset(self):
        self.states = []
        
    def savePolicy(self, rounds):
        fw = open('policy2_legr_' + str(self.lr) + '_' + str(self.exp_rate) + '_' + str(self.decay_gamma) + '_' + str(rounds) + '_' + str(self.name), 'wb')
        pickle.dump(self.states_value, fw)
        fw.close()

    def loadPolicy(self, file):
        fr = open(file,'rb')
        self.states_value = pickle.load(fr)
        fr.close()

In [5]:
class HumanPlayer:
    def __init__(self, name):
        self.name = name 
    
    def chooseAction(self, positions):
        while True:
            # for x in positions:
                # print(x)
            row1 = int(input("Input your action row1:"))
            col1 = int(input("Input your action col1:"))
            row2 = int(input("Input your action row2:"))
            col2 = int(input("Input your action col2:"))
            action = (row1, col1, row2, col2)
            if action in positions:
                return action
            else:
                sys.exit(1)
    
    # append a hash state
    def addState(self, state):
        pass
    
    # at the end of game, backpropagate and update states value
    def feedReward(self, reward):
        pass
            
    def reset(self):
        pass

### Training

In [6]:
p1 = Player("p1")
p2 = Player("p2")

st = State(p1, p2)

In [7]:
print("training...")
st.play(20000)

p1.savePolicy(20000)
p2.savePolicy(20000)

training...
Rounds 0
Rounds 10
Rounds 20
Rounds 30
Rounds 40
Rounds 50
Rounds 60
Rounds 70
Rounds 80
Rounds 90
Rounds 100
Rounds 110
Rounds 120
Rounds 130
Rounds 140
Rounds 150
Rounds 160
Rounds 170
Rounds 180
Rounds 190
Rounds 200
Rounds 210
Rounds 220
Rounds 230
Rounds 240
Rounds 250
Rounds 260
Rounds 270
Rounds 280
Rounds 290
Rounds 300
Rounds 310
Rounds 320
Rounds 330
Rounds 340
Rounds 350
Rounds 360
Rounds 370
Rounds 380
Rounds 390
Rounds 400
Rounds 410
Rounds 420
Rounds 430
Rounds 440
Rounds 450
Rounds 460
Rounds 470
Rounds 480
Rounds 490
Rounds 500
Rounds 510
Rounds 520
Rounds 530
Rounds 540
Rounds 550
Rounds 560
Rounds 570
Rounds 580
Rounds 590
Rounds 600
Rounds 610
Rounds 620
Rounds 630
Rounds 640
Rounds 650
Rounds 660
Rounds 670
Rounds 680
Rounds 690
Rounds 700
Rounds 710
Rounds 720
Rounds 730
Rounds 740
Rounds 750
Rounds 760
Rounds 770
Rounds 780
Rounds 790
Rounds 800
Rounds 810
Rounds 820
Rounds 830
Rounds 840
Rounds 850
Rounds 860
Rounds 870
Rounds 880
Rounds 890
Rounds 90

In [8]:
p1.loadPolicy("policy2_legr_0.2_0.1_0.95_20000_p1")
p2.loadPolicy("policy2_legr_0.2_0.1_0.95_20000_p2")

### Computer vs Computer

In [9]:
p1 = Player("computer1", exp_rate=0)
p1.loadPolicy("policy2_legr_0.2_0.1_0.95_1000_p1")
p2 = Player("computer2", exp_rate=0)
p2.loadPolicy("policy2_legr_0.2_0.1_0.95_1000_p2")

# p1 = HumanPlayer("human1")
# p2 = HumanPlayer("human2")

st = State(p1, p2)
st.play2()

------------------------------
|   |   |   |   |   |   | o | 
------------------------------
|   |   |   |   |   |   |   | 
------------------------------
|   | x |   |   |   |   |   | 
------------------------------
|   |   |   |   |   |   |   | 
------------------------------
|   |   |   |   |   |   |   | 
------------------------------
|   |   |   |   |   |   |   | 
------------------------------
| o |   |   |   |   |   | x | 
------------------------------
------------------------------
|   |   |   |   |   |   | o | 
------------------------------
|   |   |   |   |   |   |   | 
------------------------------
|   | x |   |   |   |   |   | 
------------------------------
|   |   |   |   |   |   |   | 
------------------------------
|   |   |   |   |   |   |   | 
------------------------------
|   |   | o |   |   |   |   | 
------------------------------
|   |   |   |   |   |   | x | 
------------------------------
------------------------------
|   |   |   |   |   |   | o | 
--------

------------------------------
| x |   |   |   |   |   | o | 
------------------------------
|   |   |   | x |   |   |   | 
------------------------------
|   |   |   |   |   |   | o | 
------------------------------
| x |   |   | o |   |   |   | 
------------------------------
| x |   |   |   |   |   | x | 
------------------------------
|   | x |   |   |   |   | x | 
------------------------------
| x | x | x | x |   |   | x | 
------------------------------
------------------------------
| x |   |   |   |   |   | o | 
------------------------------
|   |   |   | x |   |   |   | 
------------------------------
|   |   |   |   |   |   | o | 
------------------------------
| x |   |   | o |   |   |   | 
------------------------------
| x |   |   |   | o |   | x | 
------------------------------
|   | x |   |   |   |   | x | 
------------------------------
| x | x | x | x |   |   | x | 
------------------------------
------------------------------
| x |   |   |   |   |   | o | 
--------

------------------------------
| x |   | x |   | x | x | x | 
------------------------------
|   |   |   |   | x |   |   | 
------------------------------
|   |   |   |   | x |   |   | 
------------------------------
| x |   |   |   |   |   |   | 
------------------------------
| o | o | o | x |   | o |   | 
------------------------------
| o | o | o | o | o | o | o | 
------------------------------
| o | o | o |   | o | o | o | 
------------------------------
------------------------------
| x |   | x |   | x | x | x | 
------------------------------
|   |   |   |   | x |   | x | 
------------------------------
|   |   |   |   |   |   |   | 
------------------------------
| x |   |   |   |   |   |   | 
------------------------------
| o | o | o | x |   | o |   | 
------------------------------
| o | o | o | o | o | o | o | 
------------------------------
| o | o | o |   | o | o | o | 
------------------------------
------------------------------
| x |   | x |   | x | x | x | 
--------

------------------------------
| x | x |   |   |   |   | x | 
------------------------------
| x |   |   |   | x | x | x | 
------------------------------
| x |   |   |   | x | o | o | 
------------------------------
| x |   |   |   |   | o | o | 
------------------------------
| o | o |   | x |   | o | o | 
------------------------------
| o | o | o | o | o | o | o | 
------------------------------
| o | o | o | o |   | o |   | 
------------------------------
------------------------------
| x | x |   |   |   |   | x | 
------------------------------
| x |   |   |   | x | x | x | 
------------------------------
| x |   |   |   | x | o | o | 
------------------------------
| x |   |   |   |   | o | o | 
------------------------------
| o | o |   | x |   | o |   | 
------------------------------
| o | o | o | o | o | o | o | 
------------------------------
| o | o | o | o | o | o |   | 
------------------------------
------------------------------
| x | x |   |   |   | x | x | 
--------

------------------------------
| x | x | x | x | x | o | o | 
------------------------------
| x | x | x | x | o | o | o | 
------------------------------
| x |   |   | x |   | o | o | 
------------------------------
| x |   |   |   |   | o |   | 
------------------------------
| o | o |   |   |   | o | o | 
------------------------------
| o | o | o | o | o | o | o | 
------------------------------
| o | o | o | x | x | o | o | 
------------------------------
------------------------------
| x | x | x | x | x | o | o | 
------------------------------
| x | x | x | x | o | o | o | 
------------------------------
| x |   |   | x |   | o | o | 
------------------------------
| x |   |   |   |   | x |   | 
------------------------------
| o | o |   |   | x | x | o | 
------------------------------
| o | o | o | x | x | x | o | 
------------------------------
| o | o | o | x |   | o | o | 
------------------------------
------------------------------
| x | x | x | x | x | o | o | 
--------

------------------------------
| x | x | x | x | x | o | o | 
------------------------------
| x | x | x | x | x | o | o | 
------------------------------
| o | x | x |   | o | o | o | 
------------------------------
| o | x | x |   |   | o |   | 
------------------------------
| o | x | x | o | o | o | o | 
------------------------------
| x | x | x | o | o |   | o | 
------------------------------
| x | x | o | o | o |   | o | 
------------------------------
------------------------------
| x | x | x | x | x | o | o | 
------------------------------
| x | x | x | x | x | o | o | 
------------------------------
| o | x | x |   | x | o | o | 
------------------------------
| o | x | x | x |   | o |   | 
------------------------------
| o |   | x | x | x | o | o | 
------------------------------
| x | x | x | o | o |   | o | 
------------------------------
| x | x | o | o | o |   | o | 
------------------------------
------------------------------
| x | x | x | x | x | o | o | 
--------